# Machine Learning II Project

Group Members:
- Daniel Kruk 20211687
- Marcelo Junior 20211677<br>

# Customer Segmentation <br>
The main goal of this notebook is to identify relevant customer segments using statistical and machine learning techniques. In Notebook 1, we explored the customer_info and customer_basket datasets and performed some data cleaning and preprocessing. In this notebook, we will use these datasets to cluster customers based on their shared characteristics and develop targeted marketing strategies.

## Libraries
Firstly, we will import the needed libraries:

In [1]:
import pandas as pd
import numpy as np


## Preparing the data
In this step, we will prepare the data for clustering.<br>
The importation of the data will be followed by the removal of unecessary columns for this part.

In [3]:
# Dataset Importation
path = '/Users/marcelojunior/Documents/Nova IMS - Lisboa (2021 - 2024)/4º Semestre/Machine Learning II.nosync/KeizerKrukis/Project Data/'

# Customer Info Preprocessed
customer_info_ppc = pd.read_csv(path + "Customer Info PPC.csv")

# Customer Basket Preprocessed
customer_basket_ppc = pd.read_csv(path + "Customer Basket PPC.csv")

The `Custoemr Info PPC` dataset will be treated first.

In [6]:
customer_info_ppc.columns
# Loyalty_card_number is useless for this part.
# As per advice, binary variables will be dropped for the segmentation, but will be analysed a posteriori

customer_info_seg = customer_info_ppc.drop(['loyalty_card_number', 'has_loyalty_card'], axis = 1)

In [7]:
customer_info_seg

,customer_id,customer_name,highest_education,customer_gender,Age,kids_home,teens_home,number_complaints,distinct_stores_visited,lifetime_spend_groceries,...,lifetime_spend_fish,lifetime_spend_hygiene,lifetime_spend_videogames,lifetime_total_distinct_products,percentage_of_products_bought_promotion,year_first_transaction,latitude,longitude,family_size,total_spending
0,4239,Sara Moscowitz,0,0,23,0.0,0.0,0.0,2.0,91.0,...,48.0,47.0,105.0,85.0,0.192154,2017.0,38.754263,-9.163397,0.0,1545.0
1,19330,Donald Robertson,0,1,41,1.0,0.0,0.0,2.0,197.0,...,80.0,45.0,1964.0,85.0,0.098647,2015.0,38.777761,-9.156197,1.0,10219.0
2,2272,Felicia Bradshaw,3,0,36,1.0,1.0,0.0,20.0,397.0,...,170.0,46.0,5.0,408.0,0.497251,2008.0,38.785329,-9.127113,2.0,1753.0
3,25288,Emma Williams,0,0,80,3.0,4.0,1.0,2.0,5282.0,...,1068.0,617.0,1078.0,1956.0,0.206713,2010.0,38.744086,-9.156589,7.0,11195.0
4,12829,Hilda Huerta,0,0,23,0.0,0.0,0.0,2.0,82.0,...,39.0,48.0,103.0,83.0,0.200625,2015.0,38.756024,-9.162970,0.0,1370.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,625,Karen Watts,3,0,48,0.0,0.0,1.0,2.0,209.0,...,11.0,57.0,1984.0,112.0,0.096366,2008.0,38.734446,-9.171911,0.0,9730.0
29996,26447,Curtis Thompson,0,1,37,0.0,0.0,0.0,3.0,185.0,...,41.0,50.0,197.0,22.0,0.118998,2014.0,38.767070,-9.144301,0.0,2416.0
29997,182,Richard Carter,0,1,22,0.0,0.0,0.0,2.0,83.0,...,52.0,51.0,94.0,77.0,0.199432,2016.0,38.728487,-9.164140,0.0,1491.0
29998,13768,Jerry Yoder,0,1,64,1.0,0.0,1.0,20.0,391.0,...,132.0,54.0,5.0,398.0,0.494527,2006.0,38.751123,-9.182512,1.0,1608.0
